### Prepare space in flash

In [1]:
import os, subprocess

In [2]:
rootdir = "/bucket/ReiterU/DBS/data/20220321_P177_laqueus_DiI_3-8_CUBIC/"
resultdir = os.path.join(rootdir, "analysis")
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [3]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, "ants")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Prepare mask

In [4]:
import nrrd
from probemapper.io import save_nifti, load_nifti
import numpy as np

In [5]:
raw = load_nifti(os.path.join(rootdir, "analysis/536BP_10um.nii"))
mask, _ = nrrd.read(os.path.join(rootdir, "analysis/slicer_536BP/Segmentation.seg.nrrd"))
mask = np.swapaxes(mask, 0, 2)
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "536BP_10um_masked.nii"), 10, 10, 10)

In [6]:
raw = load_nifti("/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/SYTOXG_10um.nii")
mask = load_nifti("/bucket/ReiterU/DBS/atlas/O_Laqueues_v0.1/SYTOXG_10um_mask.nii.gz")
raw[mask==0] = 0
save_nifti(raw, os.path.join(tmpdir, "O_Laqueues_v0.1_10um_masked.nii"), 10, 10, 10)

### Registration

In [7]:
from probemapper.registration import ANTSParams, ANTSStage, ANTSRegisteration

In [8]:
from probemapper.registration.helper import define_initial_transformation
import numpy as np

aff = np.array([[ 1.0, 0, 0],
                [ 0, 1.0, 0],
                [ 0, 0, 1.0]], dtype='float')
transl = np.array([0, 0, 0], dtype='float')
define_initial_transformation(aff, transl, os.path.join(tmpdir, "initial.mat"))

.mat file saved as /flash/ReiterU/tmp/P177/ants/initial.mat


In [9]:
P = ANTSParams(
    use_float=False,
    interpolation="Linear",
    histogram_matching=False,
    winsorize=[0.05, 1.0],
    init_transform_fx_to_mv=os.path.join(tmpdir, "initial.mat"),
    stages=[
        ANTSStage(
            transform="Affine",
            transform_args="0.1",
            metrics="MI",
            metrics_args="1,128,Regular,0.5",
            convergence="1000x1000x1000,1e-5,15",
            shrink_factors="8x4x2",
            smoothing_sigmas="3x2x1vox"
        ),
        ANTSStage(
            transform="SyN",
            transform_args="0.1,4.0,0.2",
            metrics="CC",
            metrics_args="1,4",
            convergence="500x500x100x30,1e-6,10",
            shrink_factors="8x4x2x1",
            smoothing_sigmas="3x2x1x0vox"
        )
    ]
)

In [10]:
R = ANTSRegisteration(P, ants_path="/apps/unit/ReiterU/ANTs/2.3.5/bin", num_threads=12)
R.set_fixed_image(os.path.join(tmpdir, "O_Laqueues_v0.1_10um_masked.nii"), 10)
R.set_moving_image(os.path.join(tmpdir, "536BP_10um_masked.nii"), 10)
R.set_outputs(tmpdir)

In [11]:
R.run()

########################################
['/apps/unit/ReiterU/ANTs/2.3.5/bin/antsRegistration', '--dimensionality', '3', '--initial-moving-transform', '/flash/ReiterU/tmp/P177/ants/initial.mat', '--interpolation', 'Linear', '--use-histogram-matching', '0', '--float', '0', '--verbose', '1', '--output', '[/flash/ReiterU/tmp/P177/ants/F2M_,/flash/ReiterU/tmp/P177/ants/Warped.nii.gz,/flash/ReiterU/tmp/P177/ants/InvWarped.nii.gz]', '--transform', 'Affine[0.1]', '--metric', 'MI[/flash/ReiterU/tmp/P177/ants/O_Laqueues_v0.1_10um_masked.nii,/flash/ReiterU/tmp/P177/ants/536BP_10um_masked.nii,1,128,Regular,0.5]', '--convergence', '[1000x1000x1000,1e-5,15]', '--shrink-factors', '8x4x2', '--smoothing-sigmas', '3x2x1vox', '--transform', 'SyN[0.1,4.0,0.2]', '--metric', 'CC[/flash/ReiterU/tmp/P177/ants/O_Laqueues_v0.1_10um_masked.nii,/flash/ReiterU/tmp/P177/ants/536BP_10um_masked.nii,1,4]', '--convergence', '[500x500x100x30,1e-6,10]', '--shrink-factors', '8x4x2x1', '--smoothing-sigmas', '3x2x1x0vox']


 2DIAGNOSTIC,     6, -7.692664716365e-02, 1.797693134862e+308, 2.4179e+00, 4.6712e-02,
 2DIAGNOSTIC,     7, -7.705187150882e-02, 1.797693134862e+308, 2.4649e+00, 4.6967e-02,
 2DIAGNOSTIC,     8, -7.804133879237e-02, 1.797693134862e+308, 2.5083e+00, 4.3384e-02,
 2DIAGNOSTIC,     9, -8.093140291152e-02, 1.797693134862e+308, 2.5527e+00, 4.4423e-02,
 2DIAGNOSTIC,    10, -8.174609605131e-02, 1.797693134862e+308, 2.6017e+00, 4.8951e-02,
 2DIAGNOSTIC,    11, -8.359821791014e-02, 1.797693134862e+308, 2.6525e+00, 5.0835e-02,
 2DIAGNOSTIC,    12, -8.547534528129e-02, 1.797693134862e+308, 2.7061e+00, 5.3580e-02,
 2DIAGNOSTIC,    13, -1.020226674738e-01, 1.797693134862e+308, 2.7571e+00, 5.1012e-02,
 2DIAGNOSTIC,    14, -1.127593083081e-01, 1.797693134862e+308, 2.8109e+00, 5.3779e-02,
 2DIAGNOSTIC,    15, -1.329776611508e-01, 1.838151182609e-02, 2.8661e+00, 5.5227e-02,
 2DIAGNOSTIC,    16, -2.305662554889e-01, 3.314211305079e-02, 2.9387e+00, 7.2603e-02,
 2DIAGNOSTIC,    17, -3.115001152647e-01, 4.9

 2DIAGNOSTIC,    49, -3.866502685245e-01, 3.836717132652e-05, 2.3850e+01, 2.9543e-01,
 2DIAGNOSTIC,    50, -3.867998826075e-01, 3.953461418064e-05, 2.4150e+01, 3.0025e-01,
 2DIAGNOSTIC,    51, -3.873394789962e-01, 4.469694298350e-05, 2.4521e+01, 3.7057e-01,
 2DIAGNOSTIC,    52, -3.874497236331e-01, 4.934481669607e-05, 2.4801e+01, 2.8020e-01,
 2DIAGNOSTIC,    53, -3.877915299405e-01, 5.587973511295e-05, 2.5089e+01, 2.8774e-01,
 2DIAGNOSTIC,    54, -3.882150332841e-01, 6.475710826355e-05, 2.5546e+01, 4.5707e-01,
 2DIAGNOSTIC,    55, -3.883788676758e-01, 7.223901994714e-05, 2.5866e+01, 3.2058e-01,
 2DIAGNOSTIC,    56, -3.884831668752e-01, 7.831185968774e-05, 2.6189e+01, 3.2298e-01,
 2DIAGNOSTIC,    57, -3.886335846724e-01, 8.265127461021e-05, 2.6606e+01, 4.1620e-01,
 2DIAGNOSTIC,    58, -3.886471052288e-01, 8.359685705788e-05, 2.6882e+01, 2.7692e-01,
 2DIAGNOSTIC,    59, -3.887495458195e-01, 8.281700440340e-05, 2.7171e+01, 2.8895e-01,
 2DIAGNOSTIC,    60, -3.888544053385e-01, 8.0674199175

 2DIAGNOSTIC,    45, -4.710695973787e-01, 3.105955854948e-03, 1.3254e+02, 1.5947e+00,
 2DIAGNOSTIC,    46, -4.717387624254e-01, 3.177128668235e-03, 1.3448e+02, 1.9412e+00,
 2DIAGNOSTIC,    47, -4.739040076465e-01, 3.139616118060e-03, 1.3615e+02, 1.6639e+00,
 2DIAGNOSTIC,    48, -4.810353691366e-01, 3.044199013581e-03, 1.3826e+02, 2.1157e+00,
 2DIAGNOSTIC,    49, -4.876072137522e-01, 2.885978932035e-03, 1.4006e+02, 1.7982e+00,
 2DIAGNOSTIC,    50, -4.900063658807e-01, 2.643566304984e-03, 1.4195e+02, 1.8919e+00,
 2DIAGNOSTIC,    51, -4.904515760024e-01, 2.310033637266e-03, 1.4369e+02, 1.7401e+00,
 2DIAGNOSTIC,    52, -4.908468475678e-01, 1.906809106658e-03, 1.4524e+02, 1.5496e+00,
 2DIAGNOSTIC,    53, -4.934399193075e-01, 1.469887762567e-03, 1.4706e+02, 1.8200e+00,
 2DIAGNOSTIC,    54, -4.950209970183e-01, 1.015144434182e-03, 1.4855e+02, 1.4890e+00,
 2DIAGNOSTIC,    55, -4.958813725434e-01, 9.307639278437e-04, 1.5000e+02, 1.4521e+00,
 2DIAGNOSTIC,    56, -4.972563150117e-01, 9.0813860586

 1DIAGNOSTIC,    49, -8.710536034833e-01, 6.011210110336e-05, 5.6830e+01, 1.1476e+00,
 1DIAGNOSTIC,    50, -8.714414974604e-01, 5.696453174010e-05, 5.7940e+01, 1.1101e+00,
 1DIAGNOSTIC,    51, -8.718144131894e-01, 5.401438849533e-05, 5.9095e+01, 1.1550e+00,
 1DIAGNOSTIC,    52, -8.721682128901e-01, 5.191144904943e-05, 6.0234e+01, 1.1393e+00,
 1DIAGNOSTIC,    53, -8.725182151158e-01, 4.984054293200e-05, 6.1371e+01, 1.1375e+00,
 1DIAGNOSTIC,    54, -8.728739007391e-01, 4.798195277812e-05, 6.2438e+01, 1.0665e+00,
 1DIAGNOSTIC,    55, -8.732196632315e-01, 4.618070601285e-05, 6.3331e+01, 8.9290e-01,
 1DIAGNOSTIC,    56, -8.735480670070e-01, 4.443360414291e-05, 6.4404e+01, 1.0736e+00,
 1DIAGNOSTIC,    57, -8.738887246779e-01, 4.280007257830e-05, 6.5510e+01, 1.1058e+00,
 1DIAGNOSTIC,    58, -8.742139269885e-01, 4.129918345983e-05, 6.6643e+01, 1.1324e+00,
 1DIAGNOSTIC,    59, -8.745227812854e-01, 3.977687118122e-05, 6.7781e+01, 1.1383e+00,
 1DIAGNOSTIC,    60, -8.748322693352e-01, 3.8371517917

 1DIAGNOSTIC,    23, -8.946807541943e-01, 1.316013147625e-04, 3.4361e+02, 9.0616e+00,
 1DIAGNOSTIC,    24, -8.950826433510e-01, 1.221788857087e-04, 3.5280e+02, 9.1938e+00,
 1DIAGNOSTIC,    25, -8.954629902944e-01, 1.138062644401e-04, 3.6242e+02, 9.6192e+00,
 1DIAGNOSTIC,    26, -8.958166048266e-01, 1.060375621768e-04, 3.7153e+02, 9.1068e+00,
 1DIAGNOSTIC,    27, -8.961437813059e-01, 9.853004352027e-05, 3.8006e+02, 8.5303e+00,
 1DIAGNOSTIC,    28, -8.964470490917e-01, 9.113970173259e-05, 3.8895e+02, 8.8911e+00,
 1DIAGNOSTIC,    29, -8.967342144533e-01, 8.388573961180e-05, 3.9808e+02, 9.1289e+00,
 1DIAGNOSTIC,    30, -8.970027178185e-01, 7.678143501452e-05, 4.0676e+02, 8.6836e+00,
 1DIAGNOSTIC,    31, -8.972590235207e-01, 6.999108938792e-05, 4.1515e+02, 8.3844e+00,
 1DIAGNOSTIC,    32, -8.974965172587e-01, 6.359966016404e-05, 4.2469e+02, 9.5461e+00,
 1DIAGNOSTIC,    33, -8.977331137012e-01, 5.790161918369e-05, 4.3390e+02, 9.2095e+00,
 1DIAGNOSTIC,    34, -8.979682198153e-01, 5.3038301271

 1DIAGNOSTIC,   119, -9.067765989074e-01, 1.739652933557e-06, 1.2334e+03, 9.0910e+00,
 1DIAGNOSTIC,   120, -9.067576826366e-01, 1.574931940704e-06, 1.2424e+03, 9.0381e+00,
 1DIAGNOSTIC,   121, -9.068256434719e-01, 1.654567529485e-06, 1.2509e+03, 8.4999e+00,
 1DIAGNOSTIC,   122, -9.068030978532e-01, 1.498509503107e-06, 1.2595e+03, 8.5893e+00,
 1DIAGNOSTIC,   123, -9.068705986115e-01, 1.584809743511e-06, 1.2690e+03, 9.4430e+00,
 1DIAGNOSTIC,   124, -9.068480547212e-01, 1.429595272150e-06, 1.2783e+03, 9.3002e+00,
 1DIAGNOSTIC,   125, -9.069147218506e-01, 1.513932348570e-06, 1.2878e+03, 9.5513e+00,
 1DIAGNOSTIC,   126, -9.068923889285e-01, 1.366576604756e-06, 1.2972e+03, 9.3726e+00,
 1DIAGNOSTIC,   127, -9.069588845333e-01, 1.455396609147e-06, 1.3070e+03, 9.7959e+00,
 1DIAGNOSTIC,   128, -9.069330519212e-01, 1.303792102451e-06, 1.3167e+03, 9.7283e+00,
 1DIAGNOSTIC,   129, -9.069972679440e-01, 1.383034037775e-06, 1.3262e+03, 9.5255e+00,
 1DIAGNOSTIC,   130, -9.069727672134e-01, 1.2379351316

 1DIAGNOSTIC,    72, -9.002448544085e-01, 3.790075148750e-06, 5.7678e+03, 5.8431e+01,
 1DIAGNOSTIC,    73, -9.002665690177e-01, 3.515181940642e-06, 5.8271e+03, 5.9323e+01,
 1DIAGNOSTIC,    74, -9.003054640594e-01, 3.435507254559e-06, 5.8864e+03, 5.9249e+01,
 1DIAGNOSTIC,    75, -9.003272211947e-01, 3.245052576316e-06, 5.9484e+03, 6.1958e+01,
 1DIAGNOSTIC,    76, -9.003668619288e-01, 3.111690528182e-06, 6.0080e+03, 5.9694e+01,
 1DIAGNOSTIC,    77, -9.003886216995e-01, 2.886684967047e-06, 6.0678e+03, 5.9744e+01,
 1DIAGNOSTIC,    78, -9.004276995768e-01, 2.896899784846e-06, 6.1273e+03, 5.9553e+01,
 1DIAGNOSTIC,    79, -9.004488597437e-01, 2.838053646518e-06, 6.1852e+03, 5.7873e+01,
 1DIAGNOSTIC,    80, -9.004869019118e-01, 2.839268960539e-06, 6.2427e+03, 5.7513e+01,
 1DIAGNOSTIC,    81, -9.005071915519e-01, 2.762666430180e-06, 6.3021e+03, 5.9415e+01,
 1DIAGNOSTIC,    82, -9.005457962182e-01, 2.753256312531e-06, 6.3642e+03, 6.2064e+01,
 1DIAGNOSTIC,    83, -9.005680337254e-01, 2.6784360667

### Copy files to bucket

In [12]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/P177/ants', 'deigo:/bucket/ReiterU/DBS/data/20220321_P177_laqueus_DiI_3-8_CUBIC/analysis'], returncode=0)